# 101 example for basic understanding of OPTUNA

In [1]:

import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-01-23 21:16:21,062] A new study created in memory with name: no-name-53e7d0a7-028f-4cf5-8f85-c81d527834c3
[I 2024-01-23 21:16:21,066] Trial 0 finished with value: 87.12331360587231 and parameters: {'x': -7.3339870155187326}. Best is trial 0 with value: 87.12331360587231.
[I 2024-01-23 21:16:21,066] Trial 1 finished with value: 30.3413719166416 and parameters: {'x': 7.50830027473463}. Best is trial 1 with value: 30.3413719166416.
[I 2024-01-23 21:16:21,066] Trial 2 finished with value: 0.18554171089083965 and parameters: {'x': 2.4307455291594326}. Best is trial 2 with value: 0.18554171089083965.
[I 2024-01-23 21:16:21,066] Trial 3 finished with value: 28.965949779409954 and parameters: {'x':

{'x': 1.9877903092935247}

# Baseline Grid search for model with MNSIT dataset - PyTorch

In [ ]:
"""Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.

"""
import os
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


In [ ]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


In [1]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-01-23 22:48:34,761] A new study created in memory with name: no-name-2ac8fca7-5517-4dad-8a30-68e814974fae
[I 2024-01-23 22:48:42,344] Trial 0 finished with value: 0.4578125 and parameters: {'n_layers': 3, 'n_units_l0': 79, 'dropout_l0': 0.25549339336102284, 'n_units_l1': 114, 'dropout_l1': 0.49656211917237564, 'n_units_l2': 6, 'dropout_l2': 0.25317175001585807, 'optimizer': 'SGD', 'lr': 0.03832727259290079}. Best is trial 0 with value: 0.4578125.
[I 2024-01-23 22:48:47,747] Trial 1 finished with value: 0.70078125 and parameters: {'n_layers': 1, 'n_units_l0': 112, 'dropout_l0': 0.2274308559142771, 'optimizer': 'SGD', 'lr': 0.024002382065435257}. Best is trial 1 with value: 0.70078125.
[I 202

Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.82421875
  Params: 
    n_layers: 1
    n_units_l0: 89
    dropout_l0: 0.2795510232854767
    optimizer: Adam
    lr: 0.0025627607615811387


In [2]:
import plotly
optuna.visualization.plot_optimization_history(study)

In [3]:
optuna.visualization.plot_slice(study)